In [8]:
import requests
import html
from bs4 import BeautifulSoup
import unicodedata
from unidecode import unidecode

from datetime import datetime, timezone
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [26]:
datetime.strptime('12/12/2012 12:21:21', '%d/%m/%Y %H:%M:%S')
datetime.strptime('12/12/2012 12:21:21', '%d/%m/%Y %H:%M:%S').astimezone().isoformat()

'2012-12-12T12:21:21+01:00'

In [2]:
link = 'https://atlasarena.pl/wydarzenia'
cal_id = 'b294857be2068a08a31b53cd01ccd7fb6178edd36b21fcea91c0c14f6c01ee76@group.calendar.google.com'

In [3]:
def _get_website(link):
    response = requests.get(link)
    content = response.content.decode('utf-8')
    content = html.unescape(content)
    content = unicodedata.normalize("NFKD", content)
    soup = BeautifulSoup(content, 'html.parser')
    return soup

In [4]:
soup = _get_website(link)
dates = soup.find_all('div', {'class' : 'date'})
titles = soup.find_all('h2')

In [5]:
months_dict = {'stycznia' : '1',
               'lutego' : '2',
               'marca' : '3',
               'kwietnia' : '4',
               'maja' : '5',
               'czerwca' : '6',
               'lipca' : '7',
               'sierpnia' : '8',
               'września': '9',
               'października' : '10',
               'listopada' : '11',
               'grudnia' : '12'}

In [36]:
events_insert = []

for date, title in zip(dates, titles):
    event_metadata = {}
    event_date = date.get_text().strip().split()
    event_date[1] = months_dict.get(event_date[1].lower())
    
    start_date = '/'.join(map(str,event_date)) + ' 17:00:00'
    start_date = datetime.datetime.strptime(start_date, '%d/%m/%Y %H:%M:%S').astimezone().isoformat()
    
    end_date = '/'.join(map(str,event_date)) + ' 18:30:00'
    end_date = datetime.datetime.strptime(end_date, '%d/%m/%Y %H:%M:%S').astimezone().isoformat()
    
    event_title = title.get_text().strip()
    
    # event_metadata['summary'] = 'Atlas Arena Event'
    # event_metadata['description'] = event_title
    # event_metadata['start'] = {'dateTime' : start_date}
    # event_metadata['end'] = {'dateTime' : end_date}
    # event_metadata['reminders'] = {'useDefault' : False,
    #                                'overrides' : [
    #                                    {'method' : 'popup', 'minutes' : 15} 
                                #    ]}
    event_metadata = {'summary' : 'Atlas Arena Event',
                    'description' : event_title,
                    'start' : {'dateTime': start_date},
                    'end' : {'dateTime': end_date},
                    'reminders' : {'useDefault': False,
                                'overrides': [
                                    {'method': 'popup',
                                    'minutes': 15}]}}
    
    event_key = event_title.replace(' ', '')+ '_' + '-'.join(map(str, event_date))

    events_insert.append(event_metadata)
    
    

In [43]:
event_title.replace(' ', '')+ '_' + start_date.split('T')[0]

'LouisTomlinson_2023-11-17'

In [ ]:
import requests

In [31]:
event_title.replace(' ', '') + '_' +
'-'.join(map(str, event_date))

'LouisTomlinson_17-11-2023'

In [26]:
event_date.

['17', '11', '2023']

In [29]:
'-'.join(map(str event_date))

'17-11-2023'

In [44]:
events_insert_keys = [event['description'].replace(' ', '') + '_' + event['start']['dateTime'].split('T')[0] for event in events_insert]


In [45]:
events_insert_keys

['ChristmasTime!Concert_2022-12-17',
 'EnnioMorricone–TheOfficialConcertCelebration(ODWOŁANE)_2022-12-22',
 'KoncertpiosenekK.Krawczyka-przełożony_2023-02-24',
 'KoncertMuzykiFilmowej_2023-03-19',
 'LadyPankMTVUnplugged_2023-04-22',
 'O.S.T.R.120RapFest_2023-04-30',
 'AvrilLavigne_2023-05-02',
 'ErosRamazzotti_2023-05-09',
 'Sabaton_2023-05-12',
 'InternationalAnimeMusicFestival2023_2023-06-05',
 'MetalHammerFestival2023_2023-06-08',
 'AndréRieu_2023-06-10',
 'Scorpions_2023-06-23',
 'Disney.Księżniczki_2023-06-24',
 'Disney.Księżniczki_2023-06-25',
 'Disney.Księżniczki_2023-09-11',
 'LouisTomlinson_2023-11-17']

In [37]:
SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=50589)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    
    events_result = service.events().list(calendarId= cal_id, timeMin=now,
                                            singleEvents=True,
                                            orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
        
    events_keys = [event['description'].replace(' ', '') + '_' +
                  event['start']['dateTime'].split('T')[0]
                  for event in events]
    
    for event in events:
        print(event['summary'],"-",event['description'])
    
except HttpError as error:
    print('An error occurred: %s' % error)

Atlas Arena Event - Christmas Time! Concert
Atlas Arena Event - Ennio Morricone – The Official Concert Celebration (ODWOŁANE)
Atlas Arena Event - Koncert piosenek K. Krawczyka-przełożony
Atlas Arena Event - Koncert Muzyki Filmowej
Atlas Arena Event - Lady Pank MTV Unplugged
Atlas Arena Event - O.S.T.R. 120 Rap Fest
Atlas Arena Event - Avril Lavigne
Atlas Arena Event - Eros Ramazzotti
Atlas Arena Event - Sabaton
Atlas Arena Event - International Anime Music Festival 2023
Atlas Arena Event - Metal Hammer Festival 2023
Atlas Arena Event - André Rieu
Atlas Arena Event - Scorpions
Atlas Arena Event - Disney. Księżniczki
Atlas Arena Event - Disney. Księżniczki
Atlas Arena Event - Disney. Księżniczki
Atlas Arena Event - Louis Tomlinson


In [46]:
events_insert_keys

['ChristmasTime!Concert_2022-12-17',
 'EnnioMorricone–TheOfficialConcertCelebration(ODWOŁANE)_2022-12-22',
 'KoncertpiosenekK.Krawczyka-przełożony_2023-02-24',
 'KoncertMuzykiFilmowej_2023-03-19',
 'LadyPankMTVUnplugged_2023-04-22',
 'O.S.T.R.120RapFest_2023-04-30',
 'AvrilLavigne_2023-05-02',
 'ErosRamazzotti_2023-05-09',
 'Sabaton_2023-05-12',
 'InternationalAnimeMusicFestival2023_2023-06-05',
 'MetalHammerFestival2023_2023-06-08',
 'AndréRieu_2023-06-10',
 'Scorpions_2023-06-23',
 'Disney.Księżniczki_2023-06-24',
 'Disney.Księżniczki_2023-06-25',
 'Disney.Księżniczki_2023-09-11',
 'LouisTomlinson_2023-11-17']

In [47]:
events_keys

['ChristmasTime!Concert_2022-12-17',
 'EnnioMorricone–TheOfficialConcertCelebration(ODWOŁANE)_2022-12-22',
 'KoncertpiosenekK.Krawczyka-przełożony_2023-02-24',
 'KoncertMuzykiFilmowej_2023-03-19',
 'LadyPankMTVUnplugged_2023-04-22',
 'O.S.T.R.120RapFest_2023-04-30',
 'AvrilLavigne_2023-05-02',
 'ErosRamazzotti_2023-05-09',
 'Sabaton_2023-05-12',
 'InternationalAnimeMusicFestival2023_2023-06-05',
 'MetalHammerFestival2023_2023-06-08',
 'AndréRieu_2023-06-10',
 'Scorpions_2023-06-23',
 'Disney.Księżniczki_2023-06-24',
 'Disney.Księżniczki_2023-06-25',
 'Disney.Księżniczki_2023-09-11',
 'LouisTomlinson_2023-11-17']

b294857be2068a08a31b53cd01ccd7fb6178edd36b21fcea91c0c14f6c01ee76@group.calendar.google.com - atlas


In [48]:
for event_key, event in zip(events_insert_keys, events_insert):
    if event_key in events_keys:
        print(f'Event ID: {event_key} already in calendar')
    else:
        event = service.events().insert(calendarId = cal_id, body = event).execute()
        print(f"event created: { event['description'] }")

Event ID: ChristmasTime!Concert_2022-12-17 already in calendar
Event ID: EnnioMorricone–TheOfficialConcertCelebration(ODWOŁANE)_2022-12-22 already in calendar
Event ID: KoncertpiosenekK.Krawczyka-przełożony_2023-02-24 already in calendar
Event ID: KoncertMuzykiFilmowej_2023-03-19 already in calendar
Event ID: LadyPankMTVUnplugged_2023-04-22 already in calendar
Event ID: O.S.T.R.120RapFest_2023-04-30 already in calendar
Event ID: AvrilLavigne_2023-05-02 already in calendar
Event ID: ErosRamazzotti_2023-05-09 already in calendar
Event ID: Sabaton_2023-05-12 already in calendar
Event ID: InternationalAnimeMusicFestival2023_2023-06-05 already in calendar
Event ID: MetalHammerFestival2023_2023-06-08 already in calendar
Event ID: AndréRieu_2023-06-10 already in calendar
Event ID: Scorpions_2023-06-23 already in calendar
Event ID: Disney.Księżniczki_2023-06-24 already in calendar
Event ID: Disney.Księżniczki_2023-06-25 already in calendar
Event ID: Disney.Księżniczki_2023-09-11 alread

błąd

In [106]:
'września' == 'września'

False

In [107]:
bytes('września', encoding = 'utf-8')

b'wrze\xc5\x9bnia'

In [108]:
bytes('września', 'utf-8')

b'wrzes\xcc\x81nia'